# Implementation of Conditional GANs
Reference: https://arxiv.org/pdf/1411.1784.pdf

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

In [3]:
def to_cuda(x):
    x = Variable(x)
    if torch.cuda.is_available():
        return x.cuda()
    return x

In [4]:
def to_onehot(x, num_classes=10):
    assert isinstance(x, int) or isinstance(x.data, (torch.cuda.LongTensor, torch.LongTensor))    
    if isinstance(x, int):
        c = to_cuda(torch.zeros(1, num_classes))
        c.data[0][x] = 1
    else:
        c = to_cuda(torch.FloatTensor(x.size(0), num_classes))
        c.zero_()
        c.scatter_(1, x, 1) # dim, index, src value
    return c

In [5]:
class Discriminator(nn.Module):
    """
        Simple Discriminator w/ MLP
    """
    def __init__(self, input_size=784, label_size=10, num_classes=1):
        super(Discriminator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size+label_size, 200),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(200, num_classes),
            nn.Sigmoid(),
        )
    
    def forward(self, x, y):        
        x, y = x.view(x.size(0), -1), y.view(y.size(0), -1)
        v = torch.cat((x, y), 1) # v: [input, label] concatenated vector
        y_ = self.layer1(v)
        y_ = self.layer2(y_)
        y_ = self.layer3(y_)
        return y_

In [6]:
class Generator(nn.Module):
    """
        Simple Generator w/ MLP
    """
    def __init__(self, input_size=100, label_size=10, num_classes=784):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size+label_size, 200),
            nn.LeakyReLU(0.2),
            nn.Linear(200, 200),
            nn.LeakyReLU(0.2),
            nn.Linear(200, num_classes),
            nn.Tanh()
        )
        
    def forward(self, x, y):
        x, y = x.view(x.size(0), -1), y.view(y.size(0), -1)
        v = torch.cat((x, y), 1) # v: [input, label] concatenated vector
        y_ = self.layer(v)
        y_ = y_.view(x.size(0), 1, 28, 28)
        return y_

In [7]:
D = Discriminator().cuda()
G = Generator().cuda()
D.load_state_dict('D_c.pkl')
G.load_state_dict('G_c.pkl')

In [8]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))]
)

In [9]:
mnist = datasets.MNIST(root='../data/', train=True, transform=transform, download=True)

In [10]:
batch_size = 64
condition_size = 10

In [11]:
data_loader = DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True, drop_last=True)

In [12]:
criterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters())
G_opt = torch.optim.Adam(G.parameters())

In [34]:
max_epoch = 200 # need more than 500 epochs for training generator
step = 0
n_critic = 5 # for training more k steps about Discriminator
n_noise = 100

In [35]:
D_labels = to_cuda(torch.ones(batch_size)) # Discriminator Label to real
D_fakes = to_cuda(torch.zeros(batch_size)) # Discriminator Label to fake

for epoch in range(max_epoch):
    for idx, (images, labels) in enumerate(data_loader):
        step += 1
        # Training Discriminator
        x = to_cuda(images)
        y = to_cuda(labels)
        y = y.view(batch_size, 1)
        y = to_onehot(y)
        x_outputs = D(x, y)
        D_x_loss = criterion(x_outputs, D_labels)

        z = to_cuda(torch.randn(batch_size, n_noise))
        z_outputs = D(G(z, y), y)
        D_z_loss = criterion(z_outputs, D_fakes)
        D_loss = D_x_loss + D_z_loss
        
        D.zero_grad()
        D_loss.backward()
        D_opt.step()

        if step % n_critic == 0:
            # Training Generator
            z = to_cuda(torch.randn(batch_size, n_noise))
            z_outputs = D(G(z, y), y)
            G_loss = criterion(z_outputs, D_labels)

            D.zero_grad()
            G.zero_grad()
            G_loss.backward()
            G_opt.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {}'.format(epoch, max_epoch, step, D_loss.data[0], G_loss.data[0]))        

/home/yangyangii/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1189: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch: 1/200, Step: 1000, D Loss: 1.0565564632415771, G Loss: 1.0446336269378662
Epoch: 2/200, Step: 2000, D Loss: 1.19319486618042, G Loss: 1.1084250211715698
Epoch: 3/200, Step: 3000, D Loss: 1.1244301795959473, G Loss: 1.14289391040802
Epoch: 4/200, Step: 4000, D Loss: 1.1763558387756348, G Loss: 1.0058834552764893
Epoch: 5/200, Step: 5000, D Loss: 1.2633025646209717, G Loss: 1.0351911783218384
Epoch: 6/200, Step: 6000, D Loss: 1.2585111856460571, G Loss: 1.1054434776306152
Epoch: 7/200, Step: 7000, D Loss: 1.2069371938705444, G Loss: 0.9803129434585571
Epoch: 8/200, Step: 8000, D Loss: 1.276810646057129, G Loss: 0.8564074039459229
Epoch: 9/200, Step: 9000, D Loss: 1.1529862880706787, G Loss: 1.2688381671905518
Epoch: 10/200, Step: 10000, D Loss: 1.1455817222595215, G Loss: 0.9047998785972595
Epoch: 11/200, Step: 11000, D Loss: 1.1705278158187866, G Loss: 1.0975396633148193
Epoch: 12/200, Step: 12000, D Loss: 1.1406861543655396, G Loss: 0.8779134750366211
Epoch: 13/200, Step: 13000,

Epoch: 107/200, Step: 101000, D Loss: 1.164905309677124, G Loss: 0.9452950358390808
Epoch: 108/200, Step: 102000, D Loss: 1.145942211151123, G Loss: 1.1962862014770508
Epoch: 109/200, Step: 103000, D Loss: 1.3542085886001587, G Loss: 0.8296689391136169
Epoch: 110/200, Step: 104000, D Loss: 1.1116877794265747, G Loss: 0.8952528238296509
Epoch: 112/200, Step: 105000, D Loss: 1.2182989120483398, G Loss: 0.9432944655418396
Epoch: 113/200, Step: 106000, D Loss: 1.1311761140823364, G Loss: 0.8646923899650574
Epoch: 114/200, Step: 107000, D Loss: 1.1970000267028809, G Loss: 0.8631386756896973
Epoch: 115/200, Step: 108000, D Loss: 1.3550679683685303, G Loss: 1.1321603059768677
Epoch: 116/200, Step: 109000, D Loss: 1.1937726736068726, G Loss: 1.06623375415802
Epoch: 117/200, Step: 110000, D Loss: 1.164794921875, G Loss: 1.0883926153182983
Epoch: 118/200, Step: 111000, D Loss: 1.1578365564346313, G Loss: 1.1782543659210205
Epoch: 119/200, Step: 112000, D Loss: 1.1666350364685059, G Loss: 1.00707

In [40]:
def save_checkpoint(state, file_name='checkpoint.pth.tar'):
    torch.save(state, file_name)

In [41]:
# Saving params.
# torch.save(D.state_dict(), 'D_c.pkl')
# torch.save(G.state_dict(), 'G_c.pkl')
save_checkpoint({'epoch': epoch + 1, 'state_dict':D.state_dict(), 'optimizer' : D_opt.state_dict()}, 'D_c.pth.tar')
save_checkpoint({'epoch': epoch + 1, 'state_dict':G.state_dict(), 'optimizer' : G_opt.state_dict()}, 'G_c.pth.tar')

In [42]:
import scipy.misc

In [56]:
# generation to image
z = to_cuda(torch.randn(1, n_noise))
y = to_onehot(9)
fakeimg = G(z, y).view(28, 28)
img = fakeimg.cpu().data.numpy()
scipy.misc.toimage(img)

/home/yangyangii/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  
